<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Transformación

En este laboratorio, tu objetivo será limpiar y preparar los datos extraídos previamente de diferentes fuentes para su posterior análisis. Trabajarás con datos provenientes de la API de Red Eléctrica Española (REE) y del Instituto Nacional de Estadística (INE). Estos datos incluyen información sobre demanda y generación eléctrica a nivel provincial, así como datos demográficos y económicos. 


- Cargar los Datos Extraídos:

  - **Demanda Eléctrica:** Carga los datos de demanda eléctrica extraídos de la API de REE.

  - **Generación Eléctrica:** Carga los datos de generación eléctrica diferenciados por tipo de energía (eólica, solar, hidroeléctrica, etc.) a nivel provincial.

  - **Datos Demográficos:** Carga los datos demográficos por provincia extraídos de la web del INE.

  - **Datos Económicos:** Carga los datos del PIB por provincia obtenidos del INE.


-Limpieza de Datos:

- Datos de la API de REE:

  - **Demanda Eléctrica:**

    - **Conversión de Timestamps:** Asegúrate de que las fechas estén correctamente formateadas en `datetime`. Si es necesario, convierte los datos a un formato uniforme (por ejemplo, `YYYY-MM` para datos mensuales).

    - **Tratamiento de Valores Nulos:** Identifica y maneja los valores nulos en caso de que los haya. Puedes optar por eliminar filas con valores faltantes.

    - **Estandarización de Nombres de Provincias:** Verifica que los nombres de las provincias estén estandarizados y coincidan en todos los conjuntos de datos. Si hay inconsistencias, corrígelas.

  - **Generación Eléctrica:**

    - **Desagregación de Tecnologías:** Asegúrate de que los datos estén correctamente desglosados por tipo de energía. Revisa que los campos correspondientes a energía eólica, solar, hidroeléctrica, etc., estén bien identificados y sin errores.

    - **Normalización de Unidades:** Verifica que las unidades de energía estén estandarizadas (por ejemplo, MWh). Realiza las conversiones necesarias si se encuentran en otras unidades.

    - **Identificación de Outliers:** Revisa los valores extremos o atípicos en la generación de energía y decide si deben ser tratados o eliminados.

- Datos del INE:

  - **Datos Demográficos:**

    - **Consistencia en la Codificación de Provincias:** Asegúrate de que los nombres de las provincias en los datos demográficos coincidan con los nombres utilizados en los datos eléctricos.

    - **Revisión de Categorías:** Verifica que las categorías de edad, sexo, y nacionalidad estén correctamente etiquetadas y sean consistentes en todo el dataset.

    - **Manejo de Valores Faltantes:** Revisa la presencia de valores faltantes y decide cómo tratarlos (relleno, eliminación o sustitución).

  - **Datos Económicos:**

    - **Normalización del PIB:** Si los datos del PIB están en diferentes unidades o escalas, asegúrate de normalizarlos para que sean comparables entre provincias.

    - **Agrupación Temporal:** Si los datos económicos están disponibles en diferentes periodos temporales, agrúpalos y normalízalos para que coincidan con los datos eléctricos en términos de granularidad temporal (mensual o anual).

NOTA: Ten en cuenta que los datos los vamos a tener que insertar en una base de datos mañana, por lo que toda esta limpieza os recomendamos que la penséis para poder crear e insertar los datos mañana. 

---

### Carga de datos

In [2]:
import pandas as pd
import json
import os

from src.support import recorrer_anios, limpiar_df

---

### API Demográficos

In [3]:
tipos = ['demanda', 'generacion']
lista_df = []

for tipo in tipos:
    ruta = f'datos/{tipo}'
    lista_df.append(limpiar_df(recorrer_anios(ruta)))


### INE

In [109]:
dc_provincias = {
    'Albacete': 7,                  # Castilla - La Mancha
    'Alicante/Alacant': 15,         # Comunitat Valenciana
    'Almería': 4,                   # Andalucía
    'Araba/Álava': 10,              # País Vasco
    'Asturias': 11,                 # Principado de Asturias
    'Ávila': 8,                     # Castilla y León
    'Badajoz': 16,                  # Extremadura
    'Balears, Illes': 8743,         # Illes Balears
    'Barcelona': 9,                 # Cataluña
    'Bizkaia': 10,                  # País Vasco
    'Burgos': 8,                    # Castilla y León
    'Cáceres': 16,                  # Extremadura
    'Cádiz': 4,                     # Andalucía
    'Cantabria': 6,                 # Cantabria
    'Castellón/Castelló': 15,       # Comunitat Valenciana
    'Ciudad Real': 7,               # Castilla - La Mancha
    'Córdoba': 4,                   # Andalucía
    'Coruña, A': 17,                # Galicia
    'Cuenca': 7,                    # Castilla - La Mancha
    'Gipuzkoa': 10,                 # País Vasco
    'Girona': 9,                    # Cataluña
    'Granada': 4,                   # Andalucía
    'Guadalajara': 7,               # Castilla - La Mancha
    'Huelva': 4,                    # Andalucía
    'Huesca': 5,                    # Aragón
    'Jaén': 4,                      # Andalucía
    'León': 8,                      # Castilla y León
    'Lleida': 9,                    # Cataluña
    'Lugo': 17,                     # Galicia
    'Madrid': 13,                   # Comunidad de Madrid
    'Málaga': 4,                    # Andalucía
    'Murcia': 21,                   # Región de Murcia
    'Navarra': 14,                  # Comunidad Foral de Navarra
    'Ourense': 17,                  # Galicia
    'Palencia': 8,                  # Castilla y León
    'Palmas, Las': 8742,            # Canarias
    'Pontevedra': 17,               # Galicia
    'Rioja, La': 20,                # La Rioja
    'Salamanca': 8,                 # Castilla y León
    'Santa Cruz de Tenerife': 8742, # Canarias
    'Segovia': 8,                   # Castilla y León
    'Sevilla': 4,                   # Andalucía
    'Soria': 8,                     # Castilla y León
    'Tarragona': 9,                 # Cataluña
    'Teruel': 5,                    # Aragón
    'Toledo': 7,                    # Castilla - La Mancha
    'Valencia/València': 15,        # Comunitat Valenciana
    'Valladolid': 8,                # Castilla y León
    'Zamora': 8,                    # Castilla y León
    'Zaragoza': 5,                  # Aragón
    'Ceuta': 8744,                  # Ceuta
    'Melilla': 8745                 # Melilla
}


Datos económicos

In [11]:
ruta = 'datos/DatosEconomicos'

try:
    anios = os.listdir(ruta)

except FileNotFoundError:
    print(f"No existe la ruta: {ruta}")

lista = [] 

# Recorremos los años
for anio in anios:
    ruta_anio = os.path.join(ruta, anio)
    df = pd.read_csv(ruta_anio, sep=';', encoding='latin-1')

    lista.append(df)

In [110]:
df_econ = pd.concat(lista)

In [111]:
# Provincias
df_econ['provincia'] = df_econ['Provincias'].str.split(r'\s(\D+)', regex=True, expand=True)[1]
df_econ['provincia_id'] = df_econ['Provincias'].str.split(r'\s(\D+)', regex=True, expand=True)[0]
# Limpiar periodo (2021 tiene un (P))
df_econ['periodo'] = df_econ['periodo'].apply(lambda x: int(x.replace('(P)', '')) if type(x) == str else x)
# Convertir total a float
df_econ['Total'] = df_econ['Total'].str.replace('.', '').astype(float)
# Region_id
df_econ['region_id'] = df_econ['provincia'].apply(lambda x: dc_provincias[x])
# Reset_index
df_econ.reset_index(drop=True, inplace=True)
# Eliminar columnas sobrantes
df_econ.drop(columns='Provincias', inplace=True)

In [113]:
df_econ

,Ramas de actividad,periodo,Total,provincia,provincia_id,region_id
0,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2020,7855556.0,Albacete,02,7
1,"A. Agricultura, ganadería, silvicultura y pesca",2020,768976.0,Albacete,02,7
2,"B_E. Industrias extractivas, industria manufac...",2020,1194438.0,Albacete,02,7
3,C. - De las cuales: Industria manufacturera,2020,910902.0,Albacete,02,7
4,F. Construcción,2020,415182.0,Albacete,02,7
...,...,...,...,...,...,...
1711,"K_N. Actividades financieras y de seguros, act...",2019,258800.0,Melilla,52,8745
1712,"O_U. Administración pública y defensa, segurid...",2019,770320.0,Melilla,52,8745
1713,Valor añadido bruto total,2019,1473873.0,Melilla,52,8745
1714,Impuestos netos sobre los productos,2019,151213.0,Melilla,52,8745


Datos demográficos

In [114]:
ruta = 'datos/DatosDemograficos'

try:
    anios = os.listdir(ruta)

except FileNotFoundError:
    print(f"No existe la ruta: {ruta}")

lista = [] 

# Recorremos los años
for anio in anios:
    ruta_anio = os.path.join(ruta, anio)
    df = pd.read_csv(ruta_anio, sep=';', encoding='latin-1')

    lista.append(df)

In [148]:
df_demo = pd.concat(lista)

cond1 = df_demo['Provincias'] != 'TOTAL ESPAÑA'
cond2 = df_demo['Edad (3 grupos de edad)'] != 'TOTAL EDADES'
cond3 = df_demo['Españoles/Extranjeros'] != 'TOTAL'
cond4 = df_demo['Españoles/Extranjeros'] != r'% Extranjeros'
cond5 = df_demo['Sexo'] != 'Ambos sexos'

df_demo = df_demo[cond1 & cond2 & cond3 & cond4 & cond5]

# Provincias
df_demo['provincia'] = df_demo['Provincias'].str.split(r'\s(\D+)', regex=True, expand=True)[1]
df_demo['provincia_id'] = df_demo['Provincias'].str.split(r'\s(\D+)', regex=True, expand=True)[0]
# quitar columnas
df_demo.drop(columns='Provincias', inplace=True)
df_demo.reset_index(drop=True, inplace=True)

In [149]:
df_demo

,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total,provincia,provincia_id
0,0-15 años,Españoles,Hombres,2020,27.979,Albacete,02
1,0-15 años,Españoles,Mujeres,2020,25.953,Albacete,02
2,0-15 años,Extranjeros,Hombres,2020,2.445,Albacete,02
3,0-15 años,Extranjeros,Mujeres,2020,2.191,Albacete,02
4,16-64 años,Españoles,Hombres,2020,118.606,Albacete,02
...,...,...,...,...,...,...,...
1867,16-64 años,Extranjeros,Mujeres,2019,4.780,Melilla,52
1868,65 y más,Españoles,Hombres,2019,3.510,Melilla,52
1869,65 y más,Españoles,Mujeres,2019,4.501,Melilla,52
1870,65 y más,Extranjeros,Hombres,2019,422,Melilla,52
